In [ ]:
# run_analysis.py
from pathlib import Path
import opm_batch.config as cfg
from opm_batch import process_root
from labutils import run_with_notify

# === 在執行前修改參數 ===
cfg.B_RANGE_NT = (-32.12, 32.12)        # 磁場範圍
cfg.DISP_CENTER = 5.0                  # Dispersion 中心
cfg.WIDTHRATIO = 15.0                  # 窗口寬度比例
cfg.DISP_P0_AMPL = 0.02                # Dispersion 初始振幅

# Voigt 參數
cfg.VOIGT_USE_MODEL_SELECTION = True
cfg.VOIGT_MAXFEV = 50000

# 繪圖參數
cfg.PLOT_ENABLED = True
cfg.PLOT_DPI = 200
cfg.PLOT_FMT = "png"  

# 噪聲參數
cfg.NOISE_BAND = (5.0, 100.0)
cfg.NOISE_MODE = "lowband"

# 選擇要計算的指標
cfg.SELECT_METRICS = [
    "slope", "sloper2", 
    "noisepsd", "sensitivity",
    "lorentzfwhm", "gaussianfwhm", "voigtfwhm",
    "voigtgamma", "voigtsigma", "voigtr2"
]

# === 執行分析 ===
data_dir = Path(r"Your/data/path")
run_with_notify(
    process_root, 
    data_dir, 
    title="OPM Analysis Complete"
)

print("Analysis complete!")